<a href="https://colab.research.google.com/github/sanjuraj-c/passengers-counting-model/blob/main/passenger_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np
from tracker import Tracker

model = YOLO('yolov8s.pt')

def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        colorsBGR = [x, y]
        print(colorsBGR)

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap = cv2.VideoCapture('WhatsApp Video 2024-07-20 at 2.01.08 PM.mp4')

my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
print(class_list)
count = 0

# Define the circle area near the bus door where passengers enter
circle_center = (320, 300)  # Moved center of the circle to the right side
circle_radius = 80  # Radius of the circle
tracker = Tracker()
area_c = set()
total_capacity = 30  # Total capacity of the bus

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    # Bounding box
    a = results[0].boxes.data
    # Convert to dataframe
    px = pd.DataFrame(a).astype("float")
    # Rectangle coordinates
    list = []
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'person' in c:  # Focus on detecting persons
            list.append([x1, y1, x2, y2])
    bbox_id = tracker.update(list)
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        # Centre point of object
        cx = int((x3 + x4) / 2)
        cy = int((y3 + y4) / 2)
        distance = np.sqrt((cx - circle_center[0]) ** 2 + (cy - circle_center[1]) ** 2)
        if distance <= circle_radius:
            if id not in area_c:  # Only add the ID if it hasn't been counted yet
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2)
                cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 2)
                area_c.add(id)

    # Draw the circle area on the frame
    cv2.circle(frame, circle_center, circle_radius, (255, 255, 0), 3)
    passenger_count = len(area_c)
    cv2.putText(frame, f"Passenger Fillings: {passenger_count}", (61, 146), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
    cv2.putText(frame, f"Passenger Capacity: {total_capacity}", (61, 200), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)

    cv2.imshow("RGB", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

0: 320x640 4 persons, 550.5ms
Speed: 7.2ms preprocess, 550.5ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
174.88853

In [ ]:
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np
import math

class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0

    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])
                print(dist)

                if dist < 60:
                    self.center_points[id] = (cx, cy)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

model = YOLO('yolov8s.pt')

def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        colorsBGR = [x, y]
        print(colorsBGR)

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap = cv2.VideoCapture('WhatsApp Video 2024-07-20 at 2.01.08 PM.mp4')

my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
print(class_list)
count = 0

# Define the circle area near the bus door where passengers enter
circle_center = (320, 300)  # Moved center of the circle to the right side
circle_radius = 60  # Radius of the circle
tracker = Tracker()
area_c = set()
total_capacity = 30  # Total capacity of the bus

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    # Bounding box
    a = results[0].boxes.data
    # Convert to dataframe
    px = pd.DataFrame(a).astype("float")
    # Rectangle coordinates
    list = []
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'person' in c:  # Focus on detecting persons
            list.append([x1, y1, x2 - x1, y2 - y1])
    bbox_id = tracker.update(list)
    for bbox in bbox_id:
        x3, y3, w, h, id = bbox
        x4 = x3 + w
        y4 = y3 + h
        # Centre point of object
        cx = int((x3 + x4) / 2)
        cy = int((y3 + y4) / 2)
        distance = np.sqrt((cx - circle_center[0]) ** 2 + (cy - circle_center[1]) ** 2)
        if distance <= circle_radius:
            if id not in area_c:  # Only add the ID if it hasn't been counted yet
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2)
                cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 2)
                area_c.add(id)

    # Draw the circle area on the frame
    cv2.circle(frame, circle_center, circle_radius, (255, 255, 0), 3)
    passenger_count = len(area_c)
    cv2.putText(frame, f"Passenger Fillings: {passenger_count}", (61, 146), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
    cv2.putText(frame, f"Passenger Capacity: {total_capacity}", (61, 200), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)

    cv2.imshow("RGB", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

0: 320x640 4 persons, 577.7ms
Speed: 11.7ms preprocess, 577.7ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
137.0583